This notebook print out the area, median/std depth for the [OSTF table](https://docs.google.com/spreadsheets/d/1zynp4uQuwrW0nVP1LNd8RoC8Fd3ZLhm9NRXJJGCTUug/edit#gid=0 ); easier to print out the numbers in the order we need them.

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import os
import pylab
import pickle

In [2]:
fontsize = 16
pylab.rcParams['figure.figsize'] = (10, 6)
pylab.rcParams['axes.labelsize'] = fontsize
pylab.rcParams['xtick.labelsize'] = fontsize-2
pylab.rcParams['ytick.labelsize'] = fontsize-2
pylab.rcParams['legend.fontsize'] = fontsize
pylab.rcParams['axes.linewidth'] = 2

In [3]:
dbs = ['alt_sched', 'alt_sched_rolling', 'baseline2018a', 'colossus_2664', 'colossus_2665', 'colossus_2667', \
       'kraken_2026', 'kraken_2035', 'kraken_2036', 'kraken_2042', 'kraken_2044', 'mothra_2045', 'mothra_2049', \
       'nexus_2097', 'pontus_2002', 'pontus_2489', 'pontus_2502', 'rolling_10yrs', 'roll_mix_rolling_mix_10yrs']

In [4]:
dicts = {}
data_dir = '/global/homes/a/awan/desc/depth_data_outputs'
for dbname in dbs:
    files = [f for f in os.listdir(data_dir) if f.__contains__(dbname) and f.endswith('pickle')]
    # need to separate alt_sched from alt_sched_rolling
    if dbname=='alt_sched':
        files = [f for f in files if not f.__contains__('rolling')]
    if len(files)>1:
        raise ValueError('Have more than one files: %s'%files)
    # consider the files
    for filename in files:
        print('Read in data for %s'%dbname)
        with open('%s/%s'%(data_dir, filename), 'rb') as handle:
            dicts[dbname] = pickle.load(handle)
            
nDbs = len(dicts.keys())
print('%s dbs'%nDbs)

Read in data for alt_sched
Read in data for alt_sched_rolling
Read in data for baseline2018a
Read in data for colossus_2664
Read in data for colossus_2665
Read in data for colossus_2667
Read in data for kraken_2026
Read in data for kraken_2035
Read in data for kraken_2036
Read in data for kraken_2042
Read in data for kraken_2044
Read in data for mothra_2045
Read in data for mothra_2049
Read in data for nexus_2097
Read in data for pontus_2002
Read in data for pontus_2489
Read in data for pontus_2502
Read in data for rolling_10yrs
Read in data for roll_mix_rolling_mix_10yrs
19 dbs


**Read in area, depth stats**

In [5]:
xlabels = []
# first need to create the lists with the data each yr_cut
areas = {}
for dbname in dbs:  # loop over dbs
    if dbname.__contains__('_rolling_mix_10yrs'):
        label = dbname.split('_rolling_mix_10yrs')[0]
    else:
        label = dbname
    xlabels.append(label)
    for yr_cut in dicts[dbname]['Area (deg$^2$)'].keys():  # loop over yr cuts
        if yr_cut not in areas: areas[yr_cut] = []
        # add this db's area
        areas[yr_cut].append(dicts[dbname]['Area (deg$^2$)'][yr_cut])

In [6]:
depths = {}
stds = {}
for band in ['i']:
    for stat in ['median', 'std']:
        depth_stat = {}
        stat_key = [f for f in dicts['alt_sched'].keys() if f.__contains__(stat.capitalize())][0]
        print(stat_key)
        for dbname in dbs: # loop over dbs
            for yr_cut in areas: # loop over yr_cuts
                if yr_cut not in depth_stat:
                    depth_stat[yr_cut] = []
                # add this db's depth
                depth_stat[yr_cut].append(dicts[dbname][stat_key]['%s_%s'%(yr_cut, band)])
        
        if stat=='median': # save the median for the summary table plot
            depths = depth_stat
        else:
            stds = depth_stat

5$\sigma$ Depth: Median
5$\sigma$ Depth: Std


**Print out the markdown entries**

In [7]:
cuts = {'1yr': 'i>24.5 ; EBV<0.2', '3yr': 'i>25.0 ; EBV<0.2', '6yr': 'i>25.5 ; EBV<0.2', '10yr': 'i>26.0 ; EBV<0.2'}

for yr_cut in ['10yr']:
    print(yr_cut)
    for i, db in enumerate(dbs):
        if db.__contains__('_rolling_mix_10yrs'):
            db = db.split('_rolling_mix_10yrs')[0]
        else:
            db = db
        print('| %s | %.2f | %.2f | %.2f'%(db,  areas[yr_cut][i], depths[yr_cut][i], stds[yr_cut][i]))
    print('')

10yr
| alt_sched | 13589.95 | 26.28 | 0.11
| alt_sched_rolling | 13008.42 | 26.27 | 0.11
| baseline2018a | 14645.36 | 26.57 | 0.13
| colossus_2664 | 14667.44 | 26.57 | 0.13
| colossus_2665 | 15233.39 | 26.56 | 0.13
| colossus_2667 | 14675.15 | 26.59 | 0.13
| kraken_2026 | 14663.61 | 26.58 | 0.13
| kraken_2035 | 14663.61 | 26.56 | 0.13
| kraken_2036 | 14526.86 | 26.45 | 0.12
| kraken_2042 | 14710.61 | 26.62 | 0.13
| kraken_2044 | 19303.18 | 26.40 | 0.15
| mothra_2045 | 11685.12 | 26.41 | 0.23
| mothra_2049 | 19155.46 | 26.38 | 0.14
| nexus_2097 | 19143.92 | 26.37 | 0.14
| pontus_2002 | 19253.82 | 26.39 | 0.14
| pontus_2489 | 14595.84 | 26.53 | 0.12
| pontus_2502 | 14383.60 | 26.40 | 0.14
| rolling_10yrs | 15131.83 | 26.39 | 0.16
| roll_mix | 15159.79 | 26.38 | 0.16



In [8]:
print([float('%.2f'%f) for f in depths['10yr']])

[26.28, 26.27, 26.57, 26.57, 26.56, 26.59, 26.58, 26.56, 26.45, 26.62, 26.4, 26.41, 26.38, 26.37, 26.39, 26.53, 26.4, 26.39, 26.38]


In [9]:
print([float('%.2f'%f) for f in areas['10yr']])

[13589.95, 13008.42, 14645.36, 14667.44, 15233.39, 14675.15, 14663.61, 14663.61, 14526.86, 14710.61, 19303.18, 11685.12, 19155.46, 19143.92, 19253.82, 14595.84, 14383.6, 15131.83, 15159.79]


In [10]:
print([float('%.2f'%f) for f in stds['10yr']])

[0.11, 0.11, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.12, 0.13, 0.15, 0.23, 0.14, 0.14, 0.14, 0.12, 0.14, 0.16, 0.16]
